## Imports and configs

In [1]:
from time import sleep
import sys
sys.path.append('..')

from utils import read_txt_to_mot, convert_pickled_to_txt, write_detection_features, FeatureNormalize
from utils.models import check_encoder_output_if_normalized

In [2]:
DETECTIONS_PATH = '/mnt/c/workspace/outputs/frcnn_r18_hdet640-cam3_1hour_1.pkl'
IMG_SIZE = 64
IMG_PATH = '/mnt/c/workspace/datasets/topview/cam3_1hour_1/img1'
EXT = 'PNG'
BOX_FORMAT = 'xywh'
SKIP=None
LENGTH=None

BATCH_SIZE = 256

## Create data

In [3]:
import os
if os.path.splitext(DETECTIONS_PATH)[-1] == '.pkl':
    detections = convert_pickled_to_txt(DETECTIONS_PATH,
                                        return_df=True,
                                        skip=SKIP, length=LENGTH,
                                        destination ='./', verbose=True)
else:
    detections = read_txt_to_mot(DETECTIONS_PATH, length=14952)
detections

Increased frame ID by 1 to match MOT format.


,0,1,2,3,4,5,6,7,8,9
0,1,-1,530.346313,509.484558,114.661255,141.309082,0.997401,-1,-1,-1
1,1,-1,1251.993530,413.828430,27.150146,191.169189,0.084728,-1,-1,-1
2,1,-1,1263.261475,748.944397,15.646484,91.564331,0.081935,-1,-1,-1
3,1,-1,1264.300537,418.383514,15.276123,99.041168,0.077154,-1,-1,-1
4,1,-1,1247.036255,861.161926,32.944824,98.203369,0.075162,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
361691,14952,-1,812.116760,256.273895,115.758179,116.298767,0.996645,-1,-1,-1
361692,14952,-1,352.774597,0.981041,167.978760,147.855591,0.998795,-1,-1,-1
361693,14952,-1,952.257812,231.689972,99.287598,139.003113,0.050106,-1,-1,-1
361694,14952,-1,821.579468,361.355865,205.232422,190.212982,0.088632,-1,-1,-1


## Write features

In [4]:
import torch
from torch import nn
from torchvision import models

### resnext semi-weakly supervised learning pretrained

In [ ]:
resnext_swsl = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', 'resnext50_32x4d_swsl')


resnext_swsl.layer4 = nn.Identity()
resnext_swsl.fc = FeatureNormalize(p=2, dim=1)


write_detection_features(encoder=resnext_swsl, detections=detections,
                         destination='../sample/feats/resnext50_32x4d_swsl/',
                        img_path=IMG_PATH, batch_size=BATCH_SIZE,
                        img_size=IMG_SIZE, box_format=BOX_FORMAT, ext=EXT)

sleep(2)

Using cache found in /home/ted/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master


### resnext imagenet pretrained

In [ ]:
resnext_imgnet = models.resnext50_32x4d(pretrained=True)

resnext_imgnet.layer4 = nn.Identity()
resnext_imgnet.fc = FeatureNormalize(p=2, dim=1)

check_encoder_output_if_normalized(resnext_imgnet, device='cpu')

write_detection_features(encoder=resnext_imgnet,
                        destination='../sample/feats/resnext50_32x4d_imgnet/',
                        detections=detections,
                        img_path=IMG_PATH, batch_size=BATCH_SIZE,
                        img_size=IMG_SIZE, box_format=BOX_FORMAT, ext=EXT)

sleep(2)

### mobilenet v3

In [ ]:
mobilenet_v3 = models.mobilenet_v3_large(True)
mobilenet_v3.classifier = FeatureNormalize(p=2, dim=1)

check_encoder_output_if_normalized(mobilenet_v3, device='cpu')

write_detection_features(encoder=mobilenet_v3,
                        destination='../sample/feats/mobilenet_v3_large/',
                        detections=detections,
                        img_path=IMG_PATH, batch_size=BATCH_SIZE,
                        img_size=IMG_SIZE, box_format=BOX_FORMAT, ext=EXT)

sleep(2)

### densenet 121

In [10]:
tv_densenet = models.densenet121(True)
densenet = nn.Sequential(tv_densenet.features,
                         nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Flatten(), 
                                       FeatureNormalize(p=2, dim=1)))
check_encoder_output_if_normalized(densenet, device='cpu')
write_detection_features(encoder=densenet,
                        destination='../sample/feats/densenet121/',
                        detections=detections,
                        img_path=IMG_PATH, batch_size=BATCH_SIZE,
                        img_size=IMG_SIZE, box_format=BOX_FORMAT, ext=EXT)

sleep(2)

Output is normalized.
Convert


Features is saved at ./feats_densenet121/
Detections file is saved at ./feats_densenet121/detections.csv
